In [1]:
import numpy as np
import collections
from scipy import stats
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import JSONValueProtocol
import MRKnnTrain
import heapq
import os
import json
from itertools import islice
import ast
import csv
import pandas as pd
from datetime import datetime
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt

true = 0
false = 0

In [2]:
k = 10 
model = {}
#parse model
with open('./model_neu.json',encoding='utf-16') as src:
    for line in src:
        label_model, features_model = line.split('\t')
        features_model = features_model.replace('\n', '')
        features_model = ast.literal_eval(features_model)
        label_model = ast.literal_eval(label_model)
        model[label_model] = features_model

In [4]:
with open("./test_neu.csv") as src:
    predictor_true = 0
    predictor_false = 0
    y_pred=[]
    y_label=[]
    for line in src:
        # Extract feature set and class
        data = line.split(',')
        label = data[-1].replace('\n', '')
        y_label.append(label)
        #print(data[-1])
        features = [float(x) for x in data[:-1]]
        nearest = [] 
        count = {} 

        for cat in model:
            for point in model[cat]:
                #print(point)
                point[1:] = [float(x) for x in point[1:]]
                dis = -1*np.linalg.norm(np.array(point[1:])-np.array(features)) 
                item = tuple([dis, point[1:], cat, point[0]])
                if(len(nearest)<k):
                    # If the nearest length is less than k, append directly
                    nearest.append(item)
                    continue
                elif(len(nearest)==k):
                    # If the nearest length is equal to k, transform the nearest into a heap
                    heapq.heapify(nearest)
                if(dis > nearest[0][0]):
                    heapq.heapreplace(nearest,item)
        
        heaptemp = heapq.heappop(nearest)
        #print(range(len(nearest)))

        for i in range(len(nearest)):
            temp = heapq.heappop(nearest)
            if(temp[2] not in count):
                count[temp[2]] = 1
            else:
                count[temp[2]] += 1       
        res = max(count, key=count.get)
        y_pred.append(res)
        # Output true if the prediction is successful, otherwise false
        if(res==label):
            predictor_true += 1
        else:
            predictor_false += 1
    

SyntaxError: invalid syntax (<ipython-input-4-0dfec3486b97>, line 45)

In [27]:
print("Accuary:"+str(predictor_true/(predictor_true+predictor_false)*100)+"%")
cm=confusion_matrix(y_label, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)#display_labels=clf.classes_

accuracy2 = accuracy_score(y_label, y_pred, normalize=true)
error_rate = 1 - accuracy2

print(accuracy2, error_rate)


Accuary:0.6666666666666667%
1 0
